# ETL DATASUS - Dados de Suicídios

*Fonte: Datasus*

*Site:Kaggle*

Foco: Carregar, inspecionar, tratar valores ausentes simples, corrigir tipos de dados e carregar no banco de dados `Postgres`.

**Contexto do projeto**: Nosso objetivo é limpar e preparar uma base de dados relacionados a suicídios para criar um relatório demográfico.

## Passo 1: Configurando o ambiente

In [1]:
# Instalar as ferramentas necessárias para trabalhar com dados do tipo .csv e banco de dados
# %pip install pandas sqlalchemy psycopg2-binary -q
%pip install sqlalchemy-utils -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Importando Bibliotecas

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists,create_database
import psycopg2

#ignora avisos de atenção
import warnings
warnings.filterwarnings('ignore')

## Passo 2: E (Extract) - Extração de Dados

In [16]:
#carregando base de dados Datasus
df_datasus = pd.read_csv('base-dados/datasus_suicidio_2014_2018.csv',encoding='Windows-1252',sep=',')

#usamos o display() no lugar do print() para uma melhor visualização do DataFrame
display(df_datasus.head())

,estado,ano,CIRCOBITO,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,CODMUNRES,LOCOCOR,ASSISTMED,CAUSABAS,CAUSABAS_O,idade,mes
0,AC,2014,Suicídio,02-01-14,02-07-77,Masculino,Preta,Casado,4 a 7 anos,ATLETA PROFISSIONAL DE FUTEBOL,Rio Branco,Domicílio,Não,X700,X700,37.0,1
1,AC,2014,Suicídio,23-01-14,30-07-66,Masculino,Branca,União consensual,12 e mais,MEDICO GINECOLOGISTA E OBSTETRA,Rio Branco,Domicílio,Não,X800,X800,48.0,1
2,AC,2014,Suicídio,31-01-14,28-07-43,Masculino,Branca,NaN,NaN,0,Rio Branco,Domicílio,Não,X700,X700,71.0,1
3,AC,2014,Suicídio,05-02-14,04-07-99,Masculino,Branca,Solteiro,4 a 7 anos,ESTUDANTE,Epitaciolândia,Outros,Não,X700,X700,15.0,2
4,AC,2014,Suicídio,06-02-14,02-05-72,Masculino,Branca,NaN,NaN,0,Rio Branco,Outros,Não,X700,X700,42.0,2


In [25]:
print(f'Número de linhas e colunas: {df_datasus.shape}')
# print('\n' + '='*50 +'\n')
print("\nVamos ver as informações gerais e os tipos de dados de cada coluna:\n")
df_datasus.info()

Número de linhas e colunas: (58634, 17)

Vamos ver as informações gerais e os tipos de dados de cada coluna:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58634 entries, 0 to 58633
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   estado      58634 non-null  object 
 1   ano         58634 non-null  int64  
 2   CIRCOBITO   56606 non-null  object 
 3   DTOBITO     58634 non-null  object 
 4   DTNASC      58407 non-null  object 
 5   SEXO        58619 non-null  object 
 6   RACACOR     57298 non-null  object 
 7   ESTCIV      54305 non-null  object 
 8   ESC         45162 non-null  object 
 9   OCUP        58634 non-null  object 
 10  CODMUNRES   58634 non-null  object 
 11  LOCOCOR     58463 non-null  object 
 12  ASSISTMED   40872 non-null  object 
 13  CAUSABAS    58634 non-null  object 
 14  CAUSABAS_O  58544 non-null  object 
 15  idade       58407 non-null  float64
 16  mes         58634 non-null  int64  
dt

## Passo 3: T (Transform) - Transformação dos Dados

In [28]:
print(f'\nEstatísticas dos atributos do tipo númerico:\n {df_datasus.describe()}')


Estatísticas dos atributos do tipo númerico:
                 ano         idade           mes
count  58634.000000  58407.000000  58634.000000
mean    2016.093325     42.543308      6.585496
std        1.412665     17.378526      3.502479
min     2014.000000      0.000000      1.000000
25%     2015.000000     29.000000      3.000000
50%     2016.000000     40.000000      7.000000
75%     2017.000000     54.000000     10.000000
max     2018.000000    113.000000     12.000000


In [29]:
display(df_datasus.sample(20))

,estado,ano,CIRCOBITO,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,CODMUNRES,LOCOCOR,ASSISTMED,CAUSABAS,CAUSABAS_O,idade,mes
22965,CE,2016,Suicídio,06-04-16,25-12-45,Feminino,Branca,Solteiro,1 a 3 anos,PRODUTOR AGRICOLA POLIVALENTE,Alto Santo,Domicílio,NaN,X709,X709,71.0,4
9737,SP,2014,Suicídio,09-09-14,29-10-75,Masculino,Parda,Solteiro,8 a 11 anos,0,Diadema,Domicílio,Não,X740,X740,39.0,9
23233,CE,2016,Suicídio,26-09-16,26-06-63,Masculino,Branca,Casado,8 a 11 anos,0,Boa Viagem,Outros,Não,X709,X709,53.0,9
14450,MG,2015,Suicídio,17-03-15,02-04-65,Masculino,NaN,NaN,NaN,0,Joanésia,Domicílio,Não,X700,X700,50.0,3
50565,MG,2018,Suicídio,21-09-18,11-10-72,Masculino,Parda,Separado judicialmente,NaN,GERENTE ADMINISTRATIVO,Sete Lagoas,Outros,Não,X700,X700,46.0,9
19269,SC,2015,Suicídio,09-12-15,30-12-39,Masculino,Branca,Casado,Nenhuma,0,Tijucas,Domicílio,Não,X700,X700,76.0,12
1135,CE,2014,Suicídio,16-06-14,08-12-86,Masculino,NaN,Solteiro,1 a 3 anos,AUXILIAR DE PRODUCAO FARMACEUTICA,Fortaleza,Outros,Não,X708,X708,28.0,6
3911,MG,2014,Suicídio,16-10-14,17-06-73,Masculino,Branca,Solteiro,1 a 3 anos,DESEMPREGADO CRONICO,Poços de Caldas,Outros,Não,X709,X709,41.0,10
26337,PA,2016,Suicídio,11-06-16,20-11-98,Masculino,Parda,Solteiro,4 a 7 anos,0,Belém,Domicílio,Não,X700,X700,18.0,6
9009,SP,2014,Suicídio,03-05-14,20-05-52,Masculino,Branca,Casado,8 a 11 anos,PINTOR DE OBRAS,Osasco,Domicílio,NaN,X700,X700,62.0,5


In [ ]:
#Aplicar o describer para ver as estatísticas dos atributos do tipo texto
print('\nEstatísticas descritivas dos atributos do tipo texto:\n')
df_datasus.describe(include='object')


Estatísticas descritivas dos atributos do tipo texto:



,estado,CIRCOBITO,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,CODMUNRES,LOCOCOR,ASSISTMED,CAUSABAS,CAUSABAS_O
count,58634,56606,58634,58407,58619,57298,54305,45162,58634,58634,58463,40872,58634,58544
unique,27,4,1826,22200,2,5,5,5,1108,4769,6,2,301,527
top,SP,Suicídio,01-01-17,15-06-81,Masculino,Branca,Solteiro,8 a 11 anos,0,São Paulo,Domicílio,Não,X700,X700
freq,11256,55484,64,12,46169,29193,29408,14503,12353,2090,36070,32629,28997,26666


### 3.1 Selecionar Colunas Relevantes

In [32]:
print('\n Colunas disponíveis no DataFrame:')
df_datasus.columns


 Colunas disponíveis no DataFrame:


Index(['estado', 'ano', 'CIRCOBITO', 'DTOBITO', 'DTNASC', 'SEXO', 'RACACOR',
       'ESTCIV', 'ESC', 'OCUP', 'CODMUNRES', 'LOCOCOR', 'ASSISTMED',
       'CAUSABAS', 'CAUSABAS_O', 'idade', 'mes'],
      dtype='object')

In [47]:
# Selecionando colunas relevantes
colunas_selecionadas = [
    'DTOBITO',
    'DTNASC', 
    'SEXO', 
    'RACACOR',
    'ESTCIV', 
    'ESC', 
    'OCUP',
    'LOCOCOR',
    'idade'
]
print(f'\nColunas selecionadas:\n {colunas_selecionadas}')

df_datasus_selecionados = df_datasus[colunas_selecionadas]
display(df_datasus_selecionados.head())


Colunas selecionadas:
 ['DTOBITO', 'DTNASC', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC', 'OCUP', 'LOCOCOR', 'idade']


,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,LOCOCOR,idade
0,02-01-14,02-07-77,Masculino,Preta,Casado,4 a 7 anos,ATLETA PROFISSIONAL DE FUTEBOL,Domicílio,37.0
1,23-01-14,30-07-66,Masculino,Branca,União consensual,12 e mais,MEDICO GINECOLOGISTA E OBSTETRA,Domicílio,48.0
2,31-01-14,28-07-43,Masculino,Branca,NaN,NaN,0,Domicílio,71.0
3,05-02-14,04-07-99,Masculino,Branca,Solteiro,4 a 7 anos,ESTUDANTE,Outros,15.0
4,06-02-14,02-05-72,Masculino,Branca,NaN,NaN,0,Outros,42.0


### 3.2 Tratamento do dtype (tipo de dados)
Vamos tratar os dtype dos campos do tipo data e valores ausentes

In [48]:
# Veridicando os valores ausentes do Dataframe:
print('\nValores ausentes (null) em cada coluna:\n')
df_datasus.isnull().sum()


Valores ausentes (null) em cada coluna:



estado            0
ano               0
CIRCOBITO      2028
DTOBITO           0
DTNASC          227
SEXO             15
RACACOR        1336
ESTCIV         4329
ESC           13472
OCUP              0
CODMUNRES         0
LOCOCOR         171
ASSISTMED     17762
CAUSABAS          0
CAUSABAS_O       90
idade           227
mes               0
dtype: int64

In [49]:
df_datasus_selecionados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58634 entries, 0 to 58633
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DTOBITO  58634 non-null  object 
 1   DTNASC   58407 non-null  object 
 2   SEXO     58619 non-null  object 
 3   RACACOR  57298 non-null  object 
 4   ESTCIV   54305 non-null  object 
 5   ESC      45162 non-null  object 
 6   OCUP     58634 non-null  object 
 7   LOCOCOR  58463 non-null  object 
 8   idade    58407 non-null  float64
dtypes: float64(1), object(8)
memory usage: 4.0+ MB


In [50]:
# Convertrr para o tipo datatime as colunas de data

#Transformando a coluna DTOBITO para o tipo DateTime
df_datasus_selecionados['DTOBITO'] = pd.to_datetime(df_datasus_selecionados['DTOBITO'])

#Transformando a coluna DTNASC para o tipo DateTime
df_datasus_selecionados['DTNASC'] = pd.to_datetime(df_datasus_selecionados['DTNASC'])

In [51]:
# excluindo as linhas com valores ausentes
df_datasus_selecionados = df_datasus_selecionados.dropna(subset=['DTNASC'])

In [52]:
df_datasus_selecionados.isnull().sum()

DTOBITO        0
DTNASC         0
SEXO           3
RACACOR     1295
ESTCIV      4144
ESC        13274
OCUP           0
LOCOCOR      163
idade          0
dtype: int64

In [58]:
#Contando os valores da coluna Sexo considerando NaN também
df_datasus_selecionados['SEXO'].value_counts(dropna=False)

SEXO
Masculino        45981
Feminino         12423
Não Informado        3
Name: count, dtype: int64

In [57]:
#Tratando o campo de sexo
df_datasus_selecionados['SEXO'].fillna('Não Informado', inplace=True)

In [ ]:
# substituindo os valores ausentes (NaN) para 'Não Informado'
df_datasus_selecionados['RACACOR'].fillna('Não Informado', inplace=True)
df_datasus_selecionados['ESTCIV'].fillna('Não Informado', inplace=True)
df_datasus_selecionados['ESC'].fillna('Não Informado', inplace=True)
df_datasus_selecionados['LOCOCOR'].fillna('Não Informado', inplace=True)

df_datasus_selecionados.isnull().sum()

DTOBITO    0
DTNASC     0
SEXO       0
RACACOR    0
ESTCIV     0
ESC        0
OCUP       0
LOCOCOR    0
idade      0
dtype: int64

### 3.3 Renomeando as Colunas

In [60]:
df_datasus_selecionados.head()

,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,LOCOCOR,idade
0,2014-02-01,1977-02-07,Masculino,Preta,Casado,4 a 7 anos,ATLETA PROFISSIONAL DE FUTEBOL,Domicílio,37.0
1,2014-01-23,2066-07-30,Masculino,Branca,União consensual,12 e mais,MEDICO GINECOLOGISTA E OBSTETRA,Domicílio,48.0
2,2014-01-31,2043-07-28,Masculino,Branca,Não Informado,Não Informado,0,Domicílio,71.0
3,2014-05-02,1999-04-07,Masculino,Branca,Solteiro,4 a 7 anos,ESTUDANTE,Outros,15.0
4,2014-06-02,2072-02-05,Masculino,Branca,Não Informado,Não Informado,0,Outros,42.0


In [62]:
df_datasus_selecionados.rename(columns={
    'DTOBITO':'data_obito',
    'DTNASC':'data_nascimento',
    'SEXO': 'sexo',
    'RACACOR':'raca_cor',
    'ESTCIV': 'estado_civil',
    'ESC': 'escolaridade',
    'OCUP':'ocupacao',
    'LOCOCOR': 'local_ocorrencia'
}, inplace=True)

df_datasus_selecionados.head()

,data_obito,data_nascimento,sexo,raca_cor,estado_civil,escolaridade,ocupacao,local_ocorrencia,idade
0,2014-02-01,1977-02-07,Masculino,Preta,Casado,4 a 7 anos,ATLETA PROFISSIONAL DE FUTEBOL,Domicílio,37.0
1,2014-01-23,2066-07-30,Masculino,Branca,União consensual,12 e mais,MEDICO GINECOLOGISTA E OBSTETRA,Domicílio,48.0
2,2014-01-31,2043-07-28,Masculino,Branca,Não Informado,Não Informado,0,Domicílio,71.0
3,2014-05-02,1999-04-07,Masculino,Branca,Solteiro,4 a 7 anos,ESTUDANTE,Outros,15.0
4,2014-06-02,2072-02-05,Masculino,Branca,Não Informado,Não Informado,0,Outros,42.0


In [63]:
df_datasus_selecionados.to_csv('arquivo_tratado.csv', index=False)

# Passo 4: L (LOAD) - Carregamento no banco de dados